In [1]:
import pandas as pd
import numpy as np
import random
import logging
logging.getLogger('tensorflow').disabled = True
import time

### Import association rules

In [2]:
import pickle

antecedents = pickle.load(open('mining/antecedents.sav', 'rb'))
consequents = pickle.load(open('mining/consequents.sav', 'rb'))
confidence = pickle.load(open('mining/confidence.sav', 'rb'))

rule_count = len(antecedents)
print("\nJumlah rule : {}\n".format(rule_count))


Jumlah rule : 576



### Memilih hero radiant dan dire

#### Inisialisasi daftar hero tersedia, tim radiant, tim dire, dan daftar rekomendasi

In [3]:
def inizialize_draft_hero():
    available_hero = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,25,26,27,28,29,30,
                      31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,
                      57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,
                      83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,
                      107,108,109,110,111,112,113,114,119,120,121,123,126,128,129,135]

    radiant_team = []  # insialisasi daftar hero radiant
    dire_team = []     # insialisasi daftar hero dire 
    
    recommend_list = []
 
    most_picked = [14, 26, 8, 74, 84, 21, 35, 44, 2, 11]   # 10 most pick hero in patch 2.79
    
    return available_hero, radiant_team, dire_team, recommend_list, most_picked

#### Merekomendasikan hero

In [4]:
def recommend_hero(rad_team,available_hero,recommend_list):
    for id_hero in rad_team:
        for i in range(rule_count):
            if id_hero in antecedents[i]:
                for id_cons in consequents[i]:
                    if id_cons in available_hero:
                        if id_cons not in recommend_list:
                            recommend_list.append(id_cons)
    return recommend_list

#### Inisialisasi hero pertama team radiant

In [5]:
def random_radiant_hero(rad_team,dire_team,heroes_list):
    hero_rad = None
    while True:
        index = random.randint(0,len(heroes_list)-1)
        id = heroes_list[index]
        if id not in dire_team and id not in rad_team:
            hero_rad = id
            break
    return hero_rad
    

#### Memilih hero untuk team radiant 

In [6]:
def pick_radiant_hero(radiant_team,dire_team,heroes_av,rec_list):
    id_hero = 0
    if len(radiant_team) == 0 or len(rec_list) == 0:
        id_hero = random_radiant_hero(radiant_team,dire_team,heroes_av)
    else:
        id_hero = random_radiant_hero(radiant_team,dire_team,rec_list)
        
#     print("\n(1) Hero dipilih untuk team radiant : {}".format(id_hero))
    if id_hero in heroes_av:
        heroes_av.remove(id_hero)
    if id_hero in rec_list:
        rec_list.remove(id_hero)
    radiant_team.append(id_hero)

#     print("(1) Daftar hero team radiant      : {}".format(radiant_team))

    if len(radiant_team) < 5:
        rec_list = recommend_hero(radiant_team,heroes_av,rec_list)
#         print("(1) Rekomendasi hero              : {}\n".format(rec_list))

#### Memilih hero untuk team dire

In [7]:
def random_one_hero(dire_team,rad_team,pick_list,drop_list):
    hero_dire = None
    while True:
        index = random.randint(0,len(pick_list)-1)
        id = pick_list[index]
        if id not in dire_team and id not in rad_team:
            hero_dire = id
            break

    return hero_dire

In [8]:
def pick_dire_hero(dire_team,rad_team,heroes_av,rec_list,most_picked):
    
    picked = random.randint(1,2)
    
    if picked == 1:
        id_hero = random_one_hero(dire_team,rad_team,heroes_av,most_picked)
    else:
        id_hero = random_one_hero(dire_team,rad_team,most_picked,heroes_av)
    
    if id_hero in heroes_av:
        heroes_av.remove(id_hero)
    if id_hero in rec_list:
        rec_list.remove(id_hero)
    if id_hero in most_picked:
        most_picked.remove(id_hero)

    dire_team.append(id_hero)

#### Preprocessing

In [9]:
# mengolah daftar hero team radiant dan dire menjadi data masukan untuk MLP

def preprocessing(radiant_team,dire_team):
    input_data = pd.DataFrame()

    # urutan kolom yang digunakan pada pelatihan MLP
    heroes_id = ['1','10','100','101','102','103','104','105','106',
                 '107','108','109','11','110','111','112','113','114',
                 '119','12','120','121','123','126','128','129','13',
                 '135','14','15','16','17','18','19','2','20','21','22',
                 '23','25','26','27','28','29','3','30','31','32','33',
                 '34','35','36','37','38','39','4','40','41','42','43',
                 '44','45','46','47','48','49','5','50','51','52','53',
                 '54','55','56','57','58','59','6','60','61','62','63',
                 '64','65','66','67','68','69','7','70','71','72','73',
                 '74','75','76','77','78','79','8','80','81','82','83',
                 '84','85','86','87','88','89','9','90','91','92','93',
                 '94','95','96','97','98','99',]

    for id in heroes_id:
        input_data[id] = 0

    id_1 = radiant_team[0]
    id_2 = radiant_team[1]
    id_3 = radiant_team[2]
    id_4 = radiant_team[3]
    id_5 = radiant_team[4]
    id_6 = dire_team[0]
    id_7 = dire_team[1]
    id_8 = dire_team[2]
    id_9 = dire_team[3]
    id_10 = dire_team[4]

    input_data = input_data.append({str(id_1) : 1,
                                str(id_2) : 1, 
                                str(id_3) : 1,
                                str(id_4) : 1,
                                str(id_5) : 1,
                                str(id_6) : -1,
                                str(id_7) : -1,
                                str(id_8) : -1,
                                str(id_9) : -1,
                                str(id_10) : -1} ,
                                ignore_index=True)

    input_data = input_data.fillna(0)
    input_data = input_data.astype(int)
    
    return input_data

### Import model

In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# import model
from tensorflow import keras
model = keras.models.load_model("model/model.h5")

In [11]:
def simulation(total_simulation):
    start_time = time.time()
    win_counter = 0

    for i in range(total_simulation):

        available_hero, radiant_team, dire_team, recommend_list, most_picked = inizialize_draft_hero()

        first_pick = random.randint(1,2)

        while len(radiant_team) < 5 or len(dire_team) < 5:

        #     diplay_available_hero(available_hero)

            if first_pick == 1:
                pick_radiant_hero(radiant_team,dire_team,available_hero,recommend_list)
                pick_dire_hero(dire_team,radiant_team,available_hero,recommend_list,most_picked)
            else:
                pick_dire_hero(dire_team,radiant_team,available_hero,recommend_list,most_picked)
                pick_radiant_hero(radiant_team,dire_team,available_hero,recommend_list)

        print()

        data_test = preprocessing(radiant_team, dire_team)

        # prediksi
        array_test = data_test.to_numpy()
        # prediction = model.predict(arr).round().astype(int)
        prediction = model.predict(array_test)

        win_rate = int(prediction * 100)
        print("Simulasi {:4d} -->  {} vs {} = {}%".format(i+1,radiant_team, dire_team,win_rate))

        if win_rate > 50:
            win_counter += 1
            
    run_time = time.time() - start_time
            
    return win_counter, total_simulation, run_time

#### Run simulation

In [12]:
win_counter, total_simulation, run_time = simulation(1000)

<ipython-input-9-fe30b92e8cfb>:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  input_data[id] = 0


Simulasi    1 -->  [61, 26, 8, 84, 104] vs [85, 43, 44, 74, 2] = 84%

Simulasi    2 -->  [57, 8, 84, 104, 41] vs [74, 113, 21, 26, 14] = 59%

Simulasi    3 -->  [113, 26, 135, 8, 104] vs [36, 30, 35, 93, 21] = 48%

Simulasi    4 -->  [95, 14, 135, 74, 27] vs [84, 8, 26, 11, 83] = 59%

Simulasi    5 -->  [59, 8, 26, 104, 135] vs [21, 91, 11, 10, 62] = 77%

Simulasi    6 -->  [57, 8, 26, 84, 135] vs [53, 5, 74, 50, 14] = 70%

Simulasi    7 -->  [114, 26, 14, 84, 74] vs [83, 8, 120, 28, 63] = 61%

Simulasi    8 -->  [77, 64, 8, 84, 74] vs [94, 44, 14, 2, 11] = 81%

Simulasi    9 -->  [48, 84, 26, 74, 8] vs [14, 11, 2, 44, 35] = 77%

Simulasi   10 -->  [7, 8, 26, 74, 104] vs [114, 11, 84, 21, 2] = 54%

Simulasi   11 -->  [7, 8, 84, 26, 104] vs [22, 42, 66, 13, 91] = 68%

Simulasi   12 -->  [96, 84, 26, 135, 8] vs [35, 88, 74, 29, 94] = 60%

Simulasi   13 -->  [129, 26, 84, 74, 135] vs [14, 20, 25, 21, 2] = 65%

Simulasi   14 -->  [17, 8, 84, 104, 14] vs [35, 46, 26, 5, 11] = 58%

Simulasi 

Simulasi  118 -->  [85, 8, 84, 74, 26] vs [41, 35, 47, 21, 2] = 61%

Simulasi  119 -->  [72, 14, 74, 8, 84] vs [70, 104, 10, 26, 35] = 64%

Simulasi  120 -->  [75, 14, 8, 104, 26] vs [83, 21, 66, 112, 35] = 34%

Simulasi  121 -->  [41, 26, 74, 84, 135] vs [8, 14, 11, 35, 121] = 65%

Simulasi  122 -->  [61, 14, 74, 84, 129] vs [21, 8, 35, 26, 2] = 73%

Simulasi  123 -->  [6, 14, 26, 135, 27] vs [8, 84, 74, 89, 99] = 56%

Simulasi  124 -->  [123, 8, 14, 74, 104] vs [21, 2, 26, 84, 44] = 66%

Simulasi  125 -->  [17, 8, 26, 135, 27] vs [14, 11, 21, 2, 12] = 61%

Simulasi  126 -->  [98, 14, 26, 84, 135] vs [78, 68, 75, 45, 21] = 58%

Simulasi  127 -->  [83, 8, 74, 84, 104] vs [14, 6, 121, 26, 35] = 66%

Simulasi  128 -->  [108, 8, 104, 14, 26] vs [69, 2, 45, 49, 31] = 65%

Simulasi  129 -->  [45, 14, 8, 84, 26] vs [74, 97, 35, 17, 25] = 66%

Simulasi  130 -->  [22, 8, 26, 104, 135] vs [14, 103, 69, 74, 84] = 55%

Simulasi  131 -->  [126, 8, 84, 26, 104] vs [14, 102, 75, 105, 21] = 50%

Simu

Simulasi  234 -->  [85, 8, 14, 74, 104] vs [84, 44, 26, 53, 15] = 69%

Simulasi  235 -->  [105, 14, 8, 104, 80] vs [84, 26, 74, 11, 129] = 52%

Simulasi  236 -->  [78, 26, 74, 84, 135] vs [101, 35, 2, 11, 39] = 72%

Simulasi  237 -->  [108, 8, 84, 26, 135] vs [62, 35, 74, 11, 44] = 82%

Simulasi  238 -->  [12, 14, 8, 84, 74] vs [21, 44, 11, 48, 26] = 74%

Simulasi  239 -->  [80, 26, 74, 135, 84] vs [14, 102, 44, 38, 41] = 80%

Simulasi  240 -->  [104, 84, 8, 74, 14] vs [49, 44, 11, 26, 47] = 79%

Simulasi  241 -->  [23, 8, 104, 84, 74] vs [26, 12, 129, 43, 35] = 76%

Simulasi  242 -->  [97, 44, 14, 84, 26] vs [2, 42, 32, 21, 74] = 43%

Simulasi  243 -->  [54, 74, 14, 8, 104] vs [44, 21, 13, 2, 84] = 61%

Simulasi  244 -->  [69, 8, 104, 74, 84] vs [14, 121, 44, 21, 128] = 70%

Simulasi  245 -->  [94, 26, 14, 74, 84] vs [8, 44, 73, 21, 128] = 81%

Simulasi  246 -->  [25, 8, 26, 14, 135] vs [71, 10, 35, 2, 21] = 53%

Simulasi  247 -->  [2, 26, 84, 8, 104] vs [55, 21, 74, 11, 34] = 61%

Si

Simulasi  351 -->  [32, 26, 14, 74, 135] vs [76, 8, 2, 84, 35] = 48%

Simulasi  352 -->  [23, 14, 26, 74, 84] vs [104, 21, 35, 128, 9] = 59%

Simulasi  353 -->  [106, 14, 8, 74, 84] vs [49, 119, 44, 21, 35] = 63%

Simulasi  354 -->  [11, 8, 26, 104, 14] vs [121, 74, 2, 123, 31] = 40%

Simulasi  355 -->  [70, 74, 8, 14, 84] vs [2, 26, 23, 37, 93] = 59%

Simulasi  356 -->  [40, 8, 84, 104, 26] vs [2, 100, 11, 74, 21] = 71%

Simulasi  357 -->  [97, 44, 8, 74, 84] vs [62, 15, 14, 100, 114] = 59%

Simulasi  358 -->  [49, 8, 26, 84, 135] vs [82, 32, 95, 2, 48] = 48%

Simulasi  359 -->  [6, 14, 74, 8, 84] vs [35, 11, 20, 2, 26] = 69%

Simulasi  360 -->  [57, 8, 26, 14, 104] vs [111, 59, 74, 2, 84] = 36%

Simulasi  361 -->  [123, 10, 26, 135, 74] vs [44, 8, 2, 14, 57] = 73%

Simulasi  362 -->  [128, 8, 104, 26, 135] vs [14, 92, 74, 84, 77] = 44%

Simulasi  363 -->  [57, 8, 104, 84, 74] vs [26, 22, 35, 2, 23] = 68%

Simulasi  364 -->  [35, 14, 26, 74, 135] vs [111, 84, 8, 40, 2] = 56%

Simulasi

Simulasi  467 -->  [47, 26, 14, 74, 84] vs [31, 2, 35, 37, 69] = 65%

Simulasi  468 -->  [1, 84, 26, 14, 135] vs [65, 41, 8, 2, 74] = 71%

Simulasi  469 -->  [55, 8, 74, 104, 14] vs [107, 35, 109, 2, 126] = 67%

Simulasi  470 -->  [9, 26, 135, 8, 14] vs [4, 81, 48, 21, 100] = 48%

Simulasi  471 -->  [113, 14, 74, 8, 84] vs [21, 35, 80, 44, 2] = 70%

Simulasi  472 -->  [109, 14, 74, 84, 26] vs [11, 43, 8, 2, 57] = 70%

Simulasi  473 -->  [108, 84, 14, 74, 15] vs [44, 8, 102, 26, 35] = 82%

Simulasi  474 -->  [43, 26, 14, 8, 74] vs [52, 128, 84, 35, 76] = 38%

Simulasi  475 -->  [105, 14, 8, 104, 74] vs [64, 84, 26, 81, 35] = 35%

Simulasi  476 -->  [109, 26, 8, 135, 104] vs [74, 11, 88, 3, 25] = 59%

Simulasi  477 -->  [42, 14, 26, 135, 27] vs [74, 8, 84, 11, 2] = 58%

Simulasi  478 -->  [56, 84, 14, 8, 74] vs [26, 69, 21, 48, 40] = 65%

Simulasi  479 -->  [46, 8, 104, 26, 135] vs [112, 84, 18, 35, 98] = 42%

Simulasi  480 -->  [121, 8, 74, 84, 104] vs [2, 89, 60, 109, 100] = 67%

Simul

Simulasi  584 -->  [50, 14, 8, 74, 104] vs [103, 44, 84, 121, 26] = 70%

Simulasi  585 -->  [57, 8, 74, 26, 14] vs [21, 19, 135, 58, 103] = 61%

Simulasi  586 -->  [112, 14, 26, 135, 27] vs [8, 35, 74, 21, 84] = 58%

Simulasi  587 -->  [70, 27, 26, 74, 84] vs [11, 35, 92, 31, 64] = 54%

Simulasi  588 -->  [85, 8, 74, 26, 14] vs [114, 84, 49, 2, 35] = 43%

Simulasi  589 -->  [91, 6, 84, 74, 8] vs [35, 49, 64, 14, 109] = 59%

Simulasi  590 -->  [87, 8, 84, 14, 104] vs [54, 119, 74, 26, 2] = 59%

Simulasi  591 -->  [135, 74, 27, 14, 26] vs [21, 8, 44, 39, 35] = 78%

Simulasi  592 -->  [25, 14, 8, 104, 74] vs [51, 103, 35, 2, 21] = 51%

Simulasi  593 -->  [50, 74, 8, 26, 104] vs [14, 2, 84, 44, 3] = 63%

Simulasi  594 -->  [93, 14, 26, 74, 135] vs [21, 35, 84, 8, 42] = 48%

Simulasi  595 -->  [70, 74, 26, 8, 84] vs [54, 21, 4, 68, 11] = 58%

Simulasi  596 -->  [43, 14, 8, 26, 104] vs [74, 2, 45, 44, 21] = 51%

Simulasi  597 -->  [112, 8, 74, 14, 26] vs [21, 35, 84, 96, 60] = 46%

Simulasi 

Simulasi  700 -->  [91, 20, 8, 104, 6] vs [108, 26, 126, 97, 11] = 65%

Simulasi  701 -->  [128, 8, 74, 104, 14] vs [98, 21, 2, 75, 95] = 54%

Simulasi  702 -->  [6, 84, 8, 74, 26] vs [126, 44, 54, 70, 35] = 82%

Simulasi  703 -->  [69, 26, 14, 135, 8] vs [11, 40, 35, 74, 21] = 40%

Simulasi  704 -->  [104, 8, 26, 14, 135] vs [21, 55, 72, 32, 71] = 60%

Simulasi  705 -->  [97, 26, 135, 14, 8] vs [74, 44, 2, 25, 21] = 65%

Simulasi  706 -->  [65, 8, 84, 14, 74] vs [63, 35, 113, 31, 82] = 44%

Simulasi  707 -->  [135, 8, 74, 26, 27] vs [10, 16, 65, 84, 21] = 62%

Simulasi  708 -->  [108, 14, 26, 135, 84] vs [44, 22, 21, 8, 74] = 88%

Simulasi  709 -->  [71, 8, 74, 26, 104] vs [84, 3, 20, 14, 85] = 46%

Simulasi  710 -->  [110, 107, 8, 104, 84] vs [5, 99, 88, 70, 15] = 48%

Simulasi  711 -->  [43, 14, 26, 135, 74] vs [35, 110, 83, 42, 21] = 48%

Simulasi  712 -->  [54, 14, 26, 74, 135] vs [42, 84, 44, 35, 11] = 66%

Simulasi  713 -->  [68, 8, 74, 104, 14] vs [26, 44, 28, 84, 83] = 65%

Si

Simulasi  817 -->  [27, 8, 26, 84, 135] vs [42, 70, 2, 69, 14] = 77%

Simulasi  818 -->  [114, 26, 14, 84, 8] vs [98, 73, 83, 74, 95] = 49%

Simulasi  819 -->  [23, 8, 14, 84, 104] vs [26, 52, 74, 91, 63] = 74%

Simulasi  820 -->  [50, 74, 84, 14, 26] vs [61, 129, 15, 2, 41] = 65%

Simulasi  821 -->  [18, 14, 8, 26, 135] vs [84, 73, 119, 28, 21] = 50%

Simulasi  822 -->  [18, 14, 74, 84, 26] vs [23, 128, 8, 54, 121] = 64%

Simulasi  823 -->  [73, 14, 8, 104, 84] vs [26, 44, 10, 2, 13] = 74%

Simulasi  824 -->  [104, 74, 84, 8, 14] vs [95, 123, 21, 35, 44] = 72%

Simulasi  825 -->  [52, 26, 74, 135, 14] vs [61, 49, 2, 8, 93] = 47%

Simulasi  826 -->  [7, 8, 84, 74, 104] vs [68, 44, 112, 11, 38] = 64%

Simulasi  827 -->  [85, 94, 14, 74, 84] vs [8, 44, 26, 3, 81] = 75%

Simulasi  828 -->  [106, 14, 26, 135, 27] vs [2, 87, 74, 8, 35] = 52%

Simulasi  829 -->  [95, 26, 8, 135, 104] vs [96, 129, 14, 84, 76] = 50%

Simulasi  830 -->  [52, 14, 26, 135, 8] vs [84, 44, 70, 74, 71] = 61%

Simula

Simulasi  934 -->  [5, 8, 104, 84, 26] vs [14, 47, 21, 74, 2] = 55%

Simulasi  935 -->  [67, 84, 74, 26, 135] vs [14, 8, 93, 43, 35] = 68%

Simulasi  936 -->  [45, 8, 74, 84, 104] vs [26, 21, 14, 91, 96] = 75%

Simulasi  937 -->  [104, 84, 74, 8, 14] vs [107, 26, 112, 2, 11] = 68%

Simulasi  938 -->  [44, 84, 14, 8, 74] vs [11, 16, 91, 21, 82] = 45%

Simulasi  939 -->  [107, 8, 74, 84, 14] vs [26, 128, 21, 35, 5] = 57%

Simulasi  940 -->  [72, 14, 8, 84, 104] vs [74, 11, 1, 44, 52] = 86%

Simulasi  941 -->  [6, 26, 135, 14, 84] vs [74, 119, 105, 18, 44] = 85%

Simulasi  942 -->  [8, 84, 14, 104, 74] vs [112, 15, 43, 53, 44] = 78%

Simulasi  943 -->  [40, 14, 26, 135, 27] vs [52, 35, 8, 44, 11] = 83%

Simulasi  944 -->  [96, 84, 26, 74, 14] vs [8, 21, 38, 44, 23] = 76%

Simulasi  945 -->  [29, 8, 14, 104, 74] vs [26, 21, 84, 44, 58] = 69%

Simulasi  946 -->  [58, 8, 14, 104, 74] vs [89, 32, 94, 10, 70] = 59%

Simulasi  947 -->  [8, 26, 135, 104, 27] vs [74, 44, 107, 84, 52] = 72%

Simul

In [13]:
print("--- Simulasi sebanyak 1000 kali ---")
print("Tingkat keberhasilan sistem rekomendasi = {}%".format(int((win_counter/total_simulation)*100)))
print("Waktu simulasi = %s minutes %d seconds" % (int(run_time/60), int(run_time%60)))

--- Simulasi sebanyak 1000 kali ---
Tingkat keberhasilan sistem rekomendasi = 81%
Waktu simulasi = 2 minutes 43 seconds
